In [5]:
# Cell 1: Import Libraries
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, classification_report
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [6]:
# Cell 2: Define CNN Model (Same as your architecture)
class ECGCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 27, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, L]
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [7]:
# Cell 3: Load and Preprocess Clean Data
clean_data_path = '/kaggle/input/ecg-datasets/ecg_clean.csv'
df = pd.read_csv(clean_data_path)
print(f"Dataset loaded: {df.shape}")

# Encode labels if needed
if df['label'].dtype == 'object':
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label'])
    label_mapping = dict(zip(le.classes_, range(len(le.classes_))))
    print(f"Label encoding: {label_mapping}")
else:
    label_mapping = {i: i for i in range(5)}

# Ensure numeric
feature_cols = [col for col in df.columns if col != 'label']
for col in feature_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna()

# Convert to tensors
X = torch.tensor(df.drop('label', axis=1).values, dtype=torch.float32)
y = torch.tensor(df['label'].values, dtype=torch.long)

# Normalize
X = (X - X.mean(dim=0)) / (X.std(dim=0) + 1e-8)

print(f"\nX shape: {X.shape}, y shape: {y.shape}")
print(f"Class distribution: {np.bincount(y.numpy())}")


Dataset loaded: (100033, 217)
Label encoding: {'A': 0, 'L': 1, 'N': 2, 'R': 3, 'V': 4}

X shape: torch.Size([100033, 216]), y shape: torch.Size([100033])
Class distribution: [ 2546  8073 75028  7257  7129]


In [8]:
# Cell 4: Evaluation Function with All Metrics
def evaluate_with_metrics(model, loader, device):
    """Evaluate model and return comprehensive metrics"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Calculate metrics
    accuracy = (all_preds == all_labels).mean()
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='weighted', zero_division=0
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }


In [9]:
# Cell 5: Training Function for One Fold
def train_one_fold(train_loader, val_loader, device, num_epochs=30, patience=10):
    """Train baseline model for one fold (NO noise augmentation)"""
    model = ECGCNN(num_classes=5).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    best_val_f1 = 0.0
    patience_counter = 0
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [], 'val_f1': []
    }
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_acc = train_correct / train_total
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation phase
        val_metrics = evaluate_with_metrics(model, val_loader, device)
        val_acc = val_metrics['accuracy']
        val_f1 = val_metrics['f1_score']
        
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)
        
        # Early stopping based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load best model
    model.load_state_dict(best_model_state)
    
    return model, history, best_val_f1


In [10]:
# Cell 6: K-Fold Cross-Validation Training (Baseline - Clean Data Only)
def train_baseline_with_kfold(X, y, k=5, num_epochs=30, patience=10, batch_size=64):
    """Train baseline model using stratified k-fold cross-validation (NO noise)"""
    
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    fold_results = []
    all_histories = []
    
    print("\n" + "="*70)
    print(f"BASELINE CNN TRAINING WITH {k}-FOLD CV (CLEAN DATA ONLY - NO NOISE)")
    print("="*70 + "\n")
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n{'='*70}")
        print(f"FOLD {fold+1}/{k}")
        print(f"{'='*70}")
        
        # Split data
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        print(f"Train size: {len(X_train)}, Val size: {len(X_val)}")
        print(f"Train class dist: {np.bincount(y_train.numpy())}")
        print(f"Val class dist: {np.bincount(y_val.numpy())}")
        
        # Create datasets (NO noise augmentation for baseline)
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # Train fold
        model, history, best_val_f1 = train_one_fold(
            train_loader, val_loader, device, num_epochs, patience
        )
        
        # Evaluate on validation set
        val_metrics = evaluate_with_metrics(model, val_loader, device)
        
        print(f"\nFold {fold+1} Results:")
        print(f"  Accuracy:  {val_metrics['accuracy']:.4f}")
        print(f"  Precision: {val_metrics['precision']:.4f}")
        print(f"  Recall:    {val_metrics['recall']:.4f}")
        print(f"  F1-Score:  {val_metrics['f1_score']:.4f}")
        
        fold_results.append({
            'fold': fold + 1,
            'accuracy': val_metrics['accuracy'],
            'precision': val_metrics['precision'],
            'recall': val_metrics['recall'],
            'f1_score': val_metrics['f1_score'],
        })
        
        all_histories.append(history)
        
        # Save model from this fold
        torch.save(model.state_dict(), f'/kaggle/working/baseline_model_fold_{fold+1}.pth')
    
    # Calculate average metrics across folds
    avg_metrics = {
        'accuracy_mean': np.mean([r['accuracy'] for r in fold_results]),
        'accuracy_std': np.std([r['accuracy'] for r in fold_results]),
        'precision_mean': np.mean([r['precision'] for r in fold_results]),
        'precision_std': np.std([r['precision'] for r in fold_results]),
        'recall_mean': np.mean([r['recall'] for r in fold_results]),
        'recall_std': np.std([r['recall'] for r in fold_results]),
        'f1_score_mean': np.mean([r['f1_score'] for r in fold_results]),
        'f1_score_std': np.std([r['f1_score'] for r in fold_results]),
    }
    
    print("\n" + "="*70)
    print("BASELINE K-FOLD CROSS-VALIDATION SUMMARY")
    print("="*70)
    print(f"Accuracy:  {avg_metrics['accuracy_mean']:.4f} ± {avg_metrics['accuracy_std']:.4f}")
    print(f"Precision: {avg_metrics['precision_mean']:.4f} ± {avg_metrics['precision_std']:.4f}")
    print(f"Recall:    {avg_metrics['recall_mean']:.4f} ± {avg_metrics['recall_std']:.4f}")
    print(f"F1-Score:  {avg_metrics['f1_score_mean']:.4f} ± {avg_metrics['f1_score_std']:.4f}")
    print("="*70 + "\n")
    
    return fold_results, avg_metrics, all_histories


In [11]:
# Cell 7: Run K-Fold Cross-Validation
fold_results, avg_metrics, all_histories = train_baseline_with_kfold(
    X, y, 
    k=5, 
    num_epochs=30, 
    patience=10,
    batch_size=64
)

# Save k-fold results
results_df = pd.DataFrame(fold_results)
results_df.to_csv('/kaggle/working/baseline_kfold_results.csv', index=False)

with open('/kaggle/working/baseline_avg_metrics.json', 'w') as f:
    json.dump(avg_metrics, f, indent=2)

print("✅ Baseline K-fold training completed and results saved!")



BASELINE CNN TRAINING WITH 5-FOLD CV (CLEAN DATA ONLY - NO NOISE)


FOLD 1/5
Train size: 80026, Val size: 20007
Train class dist: [ 2037  6458 60022  5806  5703]
Val class dist: [  509  1615 15006  1451  1426]
Early stopping at epoch 23

Fold 1 Results:
  Accuracy:  0.9950
  Precision: 0.9950
  Recall:    0.9950
  F1-Score:  0.9950

FOLD 2/5
Train size: 80026, Val size: 20007
Train class dist: [ 2037  6459 60022  5805  5703]
Val class dist: [  509  1614 15006  1452  1426]

Fold 2 Results:
  Accuracy:  0.9942
  Precision: 0.9941
  Recall:    0.9942
  F1-Score:  0.9941

FOLD 3/5
Train size: 80026, Val size: 20007
Train class dist: [ 2037  6459 60022  5805  5703]
Val class dist: [  509  1614 15006  1452  1426]

Fold 3 Results:
  Accuracy:  0.9947
  Precision: 0.9946
  Recall:    0.9947
  F1-Score:  0.9946

FOLD 4/5
Train size: 80027, Val size: 20006
Train class dist: [ 2036  6458 60023  5806  5704]
Val class dist: [  510  1615 15005  1451  1425]
Early stopping at epoch 24

Fold 4 Results

In [12]:
# Cell 8: Select Best Fold Model
# Find best fold based on F1 score
best_fold_idx = np.argmax([r['f1_score'] for r in fold_results])
best_fold = best_fold_idx + 1

print(f"\nBest performing fold: {best_fold}")
print(f"F1-Score: {fold_results[best_fold_idx]['f1_score']:.4f}")

# Load best model
best_baseline_model = ECGCNN(num_classes=5).to(device)
best_baseline_model.load_state_dict(torch.load(f'/kaggle/working/baseline_model_fold_{best_fold}.pth'))

print("\n" + "="*70)
print("EVALUATING BASELINE ON NOISY TEST SETS")
print("="*70)



Best performing fold: 1
F1-Score: 0.9950

EVALUATING BASELINE ON NOISY TEST SETS


In [13]:
# Cell 9: Noise Addition Function
def add_awgn(signal, snr_db):
    """Add Additive White Gaussian Noise at specified SNR level"""
    signal_power = torch.mean(signal ** 2)
    noise_power = signal_power / (10 ** (snr_db / 10))
    noise = torch.randn_like(signal) * torch.sqrt(noise_power)
    return signal + noise


In [14]:
# Cell 10: Evaluate on Noisy Test Sets at Different SNR Levels
snr_levels = [0, 3, 6, 9, 12, 15, 18, 20]
baseline_snr_results = []

print(f"\n{'SNR (dB)':<10} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-"*70)

for snr in snr_levels:
    # Check if pre-made noisy CSV exists
    noisy_csv_path = f'/kaggle/input/ecg-datasets/ecg_noisy_{snr}db.csv'
    
    if os.path.exists(noisy_csv_path):
        # Load pre-made noisy dataset
        df_noisy = pd.read_csv(noisy_csv_path)
        
        if df_noisy['label'].dtype == 'object':
            le = LabelEncoder()
            df_noisy['label'] = le.fit_transform(df_noisy['label'])
        
        feature_cols = [col for col in df_noisy.columns if col != 'label']
        for col in feature_cols:
            df_noisy[col] = pd.to_numeric(df_noisy[col], errors='coerce')
        df_noisy = df_noisy.dropna()
        
        X_noisy = torch.tensor(df_noisy.drop('label', axis=1).values, dtype=torch.float32)
        y_noisy = torch.tensor(df_noisy['label'].values, dtype=torch.long)
        X_noisy = (X_noisy - X_noisy.mean(dim=0)) / (X_noisy.std(dim=0) + 1e-8)
    else:
        # Generate noisy data on-the-fly from clean data
        X_noisy = torch.stack([add_awgn(x, snr) for x in X], dim=0)
        y_noisy = y.clone()
    
    # Create loader
    noisy_dataset = TensorDataset(X_noisy, y_noisy)
    noisy_loader = DataLoader(noisy_dataset, batch_size=64, shuffle=False)
    
    # Evaluate
    metrics = evaluate_with_metrics(best_baseline_model, noisy_loader, device)
    
    baseline_snr_results.append({
        'snr_db': snr,
        'baseline_accuracy': metrics['accuracy'],
        'baseline_precision': metrics['precision'],
        'baseline_recall': metrics['recall'],
        'baseline_f1_score': metrics['f1_score']
    })
    
    print(f"{snr:<10} {metrics['accuracy']:<12.4f} {metrics['precision']:<12.4f} "
          f"{metrics['recall']:<12.4f} {metrics['f1_score']:<12.4f}")

print("-"*70)



SNR (dB)   Accuracy     Precision    Recall       F1-Score    
----------------------------------------------------------------------
0          0.7537       0.7252       0.7537       0.7064      
3          0.8014       0.8001       0.8014       0.7814      
6          0.8638       0.8731       0.8638       0.8608      
9          0.9135       0.9240       0.9135       0.9157      
12         0.9407       0.9508       0.9407       0.9438      
15         0.9577       0.9673       0.9577       0.9611      
18         0.9738       0.9806       0.9738       0.9761      
20         0.9836       0.9866       0.9836       0.9846      
----------------------------------------------------------------------


In [16]:
# Cell 11: Save Baseline SNR Results (FIXED)
baseline_snr_df = pd.DataFrame(baseline_snr_results)
baseline_snr_df.to_csv('/kaggle/working/baseline_snr_results.csv', index=False)

# Calculate and save summary statistics
summary_stats = {
    'model': 'Baseline CNN (Clean Data Only - K-Fold CV)',
    'k_folds': 5,
    'best_fold': int(best_fold),  # ← Convert to Python int
    'snr_levels': [int(x) for x in snr_levels],  # ← Convert to Python int list
    'clean_data_performance': {
        'accuracy_mean': float(avg_metrics['accuracy_mean']),  # ← Convert to Python float
        'accuracy_std': float(avg_metrics['accuracy_std']),
        'f1_score_mean': float(avg_metrics['f1_score_mean']),
        'f1_score_std': float(avg_metrics['f1_score_std'])
    },
    'noisy_data_performance': {
        'avg_accuracy': float(baseline_snr_df['baseline_accuracy'].mean()),
        'avg_precision': float(baseline_snr_df['baseline_precision'].mean()),
        'avg_recall': float(baseline_snr_df['baseline_recall'].mean()),
        'avg_f1_score': float(baseline_snr_df['baseline_f1_score'].mean()),
        'worst_accuracy_snr': int(baseline_snr_df.loc[baseline_snr_df['baseline_accuracy'].idxmin(), 'snr_db']),
        'worst_accuracy_value': float(baseline_snr_df['baseline_accuracy'].min()),
        'best_accuracy_snr': int(baseline_snr_df.loc[baseline_snr_df['baseline_accuracy'].idxmax(), 'snr_db']),
        'best_accuracy_value': float(baseline_snr_df['baseline_accuracy'].max())
    }
}

with open('/kaggle/working/baseline_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)

print("\n Baseline SNR evaluation results saved!")
print("\nFiles created:")
print("  - baseline_kfold_results.csv: K-fold validation metrics per fold")
print("  - baseline_avg_metrics.json: Average metrics across folds")
print("  - baseline_snr_results.csv: Performance metrics at each SNR level")
print("  - baseline_summary.json: Complete summary statistics")
print("  - baseline_model_fold_1.pth to baseline_model_fold_5.pth: Saved models")



 Baseline SNR evaluation results saved!

Files created:
  - baseline_kfold_results.csv: K-fold validation metrics per fold
  - baseline_avg_metrics.json: Average metrics across folds
  - baseline_snr_results.csv: Performance metrics at each SNR level
  - baseline_summary.json: Complete summary statistics
  - baseline_model_fold_1.pth to baseline_model_fold_5.pth: Saved models


In [17]:
# Cell 12: Display Complete Summary
print("\n" + "="*70)
print("COMPLETE BASELINE CNN SUMMARY")
print("="*70)

print("\n1. K-Fold Cross-Validation Results (Clean Data):")
print(f"   Accuracy:  {avg_metrics['accuracy_mean']:.4f} ± {avg_metrics['accuracy_std']:.4f}")
print(f"   Precision: {avg_metrics['precision_mean']:.4f} ± {avg_metrics['precision_std']:.4f}")
print(f"   Recall:    {avg_metrics['recall_mean']:.4f} ± {avg_metrics['recall_std']:.4f}")
print(f"   F1-Score:  {avg_metrics['f1_score_mean']:.4f} ± {avg_metrics['f1_score_std']:.4f}")

print("\n2. Average Performance on Noisy Data (0-20 dB SNR):")
print(f"   Accuracy:  {summary_stats['noisy_data_performance']['avg_accuracy']:.4f}")
print(f"   Precision: {summary_stats['noisy_data_performance']['avg_precision']:.4f}")
print(f"   Recall:    {summary_stats['noisy_data_performance']['avg_recall']:.4f}")
print(f"   F1-Score:  {summary_stats['noisy_data_performance']['avg_f1_score']:.4f}")

print("\n3. Performance Range on Noisy Data:")
print(f"   Best:  {summary_stats['noisy_data_performance']['best_accuracy_value']:.4f} at SNR {summary_stats['noisy_data_performance']['best_accuracy_snr']} dB")
print(f"   Worst: {summary_stats['noisy_data_performance']['worst_accuracy_value']:.4f} at SNR {summary_stats['noisy_data_performance']['worst_accuracy_snr']} dB")
print(f"   Degradation: {(summary_stats['noisy_data_performance']['best_accuracy_value'] - summary_stats['noisy_data_performance']['worst_accuracy_value']) * 100:.2f}%")

print("\n4. Best Fold Model Used:")
print(f"   Fold: {best_fold}")
print(f"   F1-Score: {fold_results[best_fold_idx]['f1_score']:.4f}")

print("\n" + "="*70)
print("\n✅ Baseline training and evaluation complete!")
print("\nNext Steps:")
print("1. Run the k-fold training script WITH noise augmentation")
print("2. Use the visualization script to compare baseline vs noise-augmented results")
print("="*70)



COMPLETE BASELINE CNN SUMMARY

1. K-Fold Cross-Validation Results (Clean Data):
   Accuracy:  0.9942 ± 0.0005
   Precision: 0.9942 ± 0.0005
   Recall:    0.9942 ± 0.0005
   F1-Score:  0.9942 ± 0.0005

2. Average Performance on Noisy Data (0-20 dB SNR):
   Accuracy:  0.8985
   Precision: 0.9010
   Recall:    0.8985
   F1-Score:  0.8913

3. Performance Range on Noisy Data:
   Best:  0.9836 at SNR 20 dB
   Worst: 0.7537 at SNR 0 dB
   Degradation: 22.99%

4. Best Fold Model Used:
   Fold: 1
   F1-Score: 0.9950


✅ Baseline training and evaluation complete!

Next Steps:
1. Run the k-fold training script WITH noise augmentation
2. Use the visualization script to compare baseline vs noise-augmented results


In [18]:
# Cell 13: Display Detailed Results Table
print("\n" + "="*70)
print("DETAILED BASELINE RESULTS BY SNR LEVEL")
print("="*70 + "\n")

print(baseline_snr_df.to_string(index=False))
print("\n" + "="*70)



DETAILED BASELINE RESULTS BY SNR LEVEL

 snr_db  baseline_accuracy  baseline_precision  baseline_recall  baseline_f1_score
      0           0.753711            0.725188         0.753711           0.706447
      3           0.801356            0.800076         0.801356           0.781382
      6           0.863805            0.873064         0.863805           0.860777
      9           0.913459            0.923952         0.913459           0.915745
     12           0.940750            0.950837         0.940750           0.943801
     15           0.957664            0.967308         0.957664           0.961088
     18           0.973759            0.980641         0.973759           0.976147
     20           0.983635            0.986614         0.983635           0.984634

